# 6. Advanced RAG


In [28]:
import os
from pathlib import Path
from dotenv import load_dotenv

# rag_ai_agent_book/.envから環境変数を読み込む
dotenv_path = os.path.join(os.getcwd(), 'rag_ai_agent_book', '.env')

load_dotenv(dotenv_path)

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "rag-ai-agent-book"
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

## 6.2. ハンズオンの準備


In [26]:
# langchain_communityパッケージからGitLoaderをインポート
from langchain_community.document_loaders import GitLoader

# .mdxファイルのみを対象とするフィルタ関数を定義
def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")

# GitリポジトリからドキュメントをロードするためのGitLoaderインスタンスを作成
loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",  # クローン元のリポジトリURL
    repo_path="./langchain",                                # ローカルにクローンするパス
    branch="master",                                        # 対象ブランチ
    file_filter=file_filter,                                # ファイルフィルタ関数
)

# GitLoaderでドキュメントをロード
documents = loader.load()
# ロードしたドキュメント数を出力
print(len(documents))

# langchain_chromaからChroma、langchain_openaiからOpenAIEmbeddingsをインポート
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

# OpenAIの埋め込みモデル（text-embedding-3-small）を初期化
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
# ドキュメントと埋め込みモデルからChromaベクトルDBを作成
db = Chroma.from_documents(documents, embeddings)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


418


In [29]:
# 文字列出力用のパーサーをインポート
from langchain_core.output_parsers import StrOutputParser
# チャットプロンプトテンプレートをインポート
from langchain_core.prompts import ChatPromptTemplate
# 入力をそのまま渡すためのランナブルパススルーをインポート
from langchain_core.runnables import RunnablePassthrough
# OpenAIのチャットモデルをインポート
from langchain_openai import ChatOpenAI

# プロンプトテンプレートを作成（from_templateメソッドを使用）
prompt=ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。
文脈:"""
{context}
"""
質問:{question}
''')

# OpenAIのgpt-4o-miniモデルを初期化（temperature=0で決定論的出力）
model=ChatOpenAI(model="gpt-4o-mini", temperature=0)
# ベクトルDBからレトリバーを作成
retriever=db.as_retriever()
# チェーンを構築（questionはそのまま渡し、contextはretriverで取得）
chain={
	"question":RunnablePassthrough(), # 質問はそのまま渡す
	"context":retriever,              # contextにはretriverを指定
}|prompt|model|StrOutputParser()     # プロンプト→モデル→出力パーサーの順にパイプ

# チェーンに質問を投げて実行
chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下の3つの主要なステージがあります。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築します。LangGraphを利用することで、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **プロダクション化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできるようにします。\n\n3. **デプロイメント**: LangGraphアプリケーションをプロダクション対応のAPIやアシスタントに変換します。\n\nLangChainは、LLMや関連技術（埋め込みモデルやベクターストアなど）に対する標準インターフェースを実装し、数百のプロバイダーと統合しています。また、複数のオープンソースライブラリで構成されており、ユーザーは簡単にアプリケーションを構築、デプロイ、管理することができます。'

## 6.3. 検索クエリの工夫


### HyDE（Hypothetical Document Embeddings）


In [ ]:
# HyDe(HypotheticalDocumentEmbeddings)を利用したChatシステム
# 質問に対する類似ドキュメントではなく、仮回答に対応するドキュメントを利用する

# プロンプトのテンプレートを定義
hypothetical_prompt=ChatPromptTemplate.from_template("""\
次の質問に回答する一文を書いてください。
質問:{question}
 """)
# プロンプト、モデル、出力パーサーをパイプでつなぎ、仮回答生成チェーンを作成
hypothetical_chain=hypothetical_prompt|model|StrOutputParser()

# HyDE方式のRAGチェーンを作成（質問はそのまま、contextは仮回答→retriverで取得）
hyde_rag_chain={
	"question":RunnablePassthrough(), # 質問はそのまま渡す
	"context":hypothetical_chain|retriever # contextには仮回答をretriverに渡して取得
}|prompt|model|StrOutputParser() # プロンプト→モデル→出力パーサーの順にパイプ

# チェーンに質問を投げて実行
hyde_rag_chain.invoke("LangChainの概要を教えて")


'LangChainは、開発者が推論を行うアプリケーションを簡単に構築できるようにすることを目的としたPythonパッケージおよび企業です。元々は単一のオープンソースパッケージとして始まりましたが、現在は企業とエコシステム全体に進化しています。\n\nLangChainの主な特徴には以下の3つがあります：\n\n1. **標準化されたコンポーネントインターフェース**: AIアプリケーションに必要なさまざまなモデルや関連コンポーネントのAPIが多様化しているため、開発者がプロバイダー間で切り替えたり、コンポーネントを組み合わせたりするのが難しくなっています。LangChainは、主要なコンポーネントのための標準インターフェースを提供し、プロバイダー間の切り替えを容易にします。\n\n2. **オーケストレーション**: アプリケーションが複雑になるにつれて、複数のコンポーネントやモデルを効率的に接続する必要があります。LangChainは、これらの要素を制御フローに組み込むためのオーケストレーション機能を提供します。\n\n3. **可観測性と評価**: アプリケーションが複雑になると、その内部で何が起こっているのかを理解するのが難しくなります。LangChainは、開発者がアプリケーションを監視し、迅速に評価を行うためのツールを提供します。\n\nLangChainは、個々のコンポーネントを単独で使用することも可能であり、開発者は自分のユースケースに最適なコンポーネントを選択できます。また、LangGraphというライブラリを使用することで、複雑なアプリケーションのオーケストレーションが可能になります。さらに、LangSmithというプラットフォームを通じて、アプリケーションの可観測性と評価をサポートしています。'

### 複数の検索クエリの生成


In [ ]:
# pydanticのBaseModelとFieldをインポート
from pydantic import BaseModel, Field

# 検索クエリのリストを格納するための出力モデルを定義
class QueryGenerationOutput(BaseModel):
	queries:list[str]=Field(...,description="検索クエリのリスト") # 検索クエリのリストを格納

# 検索クエリ生成用のプロンプトテンプレートを作成
query_generation_prompt=ChatPromptTemplate.from_template("""\
質問に対してベクターデータベースから関連文章を検索するために、
3つの異なる検索クエリを生成せよ。
距離ベースの類似性検索の限界を克服するために、
ユーザーの質問に対して複数の視点を提供することが目標である。
質問:{question}
""") # プロンプトテンプレートの内容を指定

# プロンプト→モデル→出力整形のチェーンを構築
query_generation_chain=(
	query_generation_prompt
	|model.with_structured_output(QueryGenerationOutput) # モデルの出力を構造化
	|(lambda x:x.queries) # queriesリストのみを抽出
)

# 複数クエリによるRAGチェーンを定義
multi_query_rag_chain={
	"question":RunnablePassthrough(), # 質問をそのまま渡す
	"context":query_generation_chain|retriever.map() # 生成したクエリで検索を実行
}|prompt|model|StrOutputParser() # プロンプト→モデル→出力パース

# チェーンに質問を投げて実行
multi_query_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。LangChainは、開発、運用、デプロイの各段階を簡素化することを目的としています。以下はLangChainの主な特徴です。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。また、LangGraphを利用して、状態を持つエージェントを構築し、ストリーミングや人間の介入をサポートします。\n\n2. **運用化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを本番環境向けのAPIやアシスタントに変換できます。\n\nLangChainは、さまざまなプロバイダーと統合できる標準インターフェースを実装しており、開発者が異なるコンポーネントを簡単に切り替えたり、組み合わせたりできるようにしています。また、複雑なアプリケーションのオーケストレーションを行うためのLangGraphや、アプリケーションの可視化と評価を行うLangSmithといったツールも提供しています。\n\n全体として、LangChainはAIアプリケーションの開発を効率化し、開発者がより迅速に高品質なアプリケーションを構築できるようにすることを目指しています。'

## 6.4. 検索後の工夫


### RAG Fusion


In [ ]:
# 検索結果をスコアリングして成績の良い順に並び替えて、AI回答精度を上げるという仕組み

# langchain_core.documents から Document クラスをインポート
from langchain_core.documents import Document

# Reciprocal Rank Fusion（RRF）を実装する関数を定義
def reciprocal_rank_fusion(retriever_outputs:list[list[Document]],k: int=60)->list[str]:
	# 各ドキュメントの内容とスコアを格納する辞書を初期化
	content_score_mapping={}
	# 各クエリの検索結果リストに対してループ
	for docs in retriever_outputs:
		# 各ドキュメントとその順位に対してループ
		for rank,doc in enumerate(docs):
			# ドキュメントの本文を取得
			content=doc.page_content
			# 初めて出現した内容ならスコアを0で初期化
			if content not in content_score_mapping:
				content_score_mapping[content]=0
			# RRFのスコア（1/(順位+k)）を加算
			content_score_mapping[content]+=1/(rank+k)

	# スコアの降順でソート
	ranked=sorted(content_score_mapping.items(),key=lambda x:x[1], reverse=True)
	# ソート済みの内容のみをリストで返す
	return [content for content, _ in ranked]

rag_fusion_chain={
	"question":RunnablePassthrough(),
	"context":query_generation_chain|retriever.map()|reciprocal_rank_fusion
}|prompt|model|StrOutputParser()

rag_fusion_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。LangChainは、開発者がアプリケーションを簡単に構築できるようにすることを目的としており、オープンソースのコンポーネントやサードパーティの統合を利用して、LLMアプリケーションのライフサイクルの各段階を簡素化します。\n\n主な特徴には以下が含まれます：\n\n1. **標準化されたコンポーネントインターフェース**: 様々なAIアプリケーションのためのモデルや関連コンポーネントの多様なAPIを統一し、開発者がプロバイダー間で簡単に切り替えられるようにします。\n\n2. **オーケストレーション**: 複数のコンポーネントやモデルを効率的に接続し、複雑なアプリケーションを構築するための制御フローを提供します。\n\n3. **可観測性と評価**: アプリケーションの複雑さが増す中で、何が起こっているのかを理解しやすくし、開発者が迅速に評価を行えるようにします。\n\nLangChainは、個々のコンポーネントを単独で使用することも可能で、開発者は自分のユースケースに最適なコンポーネントを選択できます。また、LangGraphというオーケストレーションフレームワークを使用することで、状態を持つエージェントや複雑なアプリケーションを構築することができます。さらに、LangSmithを利用することで、アプリケーションのトレースや評価を行い、継続的な最適化とデプロイを支援します。'

### Cohere のリランクモデルを使用する準備


In [ ]:
os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY")

### Cohere のリランクモデルの導入


In [ ]:
from typing import Any# typingモジュールからAny型をインポート
from langchain_cohere import CohereRerank# langchain_cohereパッケージからCohereRerankクラスをインポート
from langchain_core.documents import Document# langchain_core.documentsからDocumentクラスをインポート

# rerank関数を定義。入力は辞書型inpと、上位N件を指定するtop_n（デフォルト3）
def rerank(inp:dict[str,Any], top_n:int=3)->list[Document]:
	# inpから質問文を取得
	question=inp["question"]
	# inpからドキュメントリストを取得
	documents=inp["documents"]

	# CohereRerankインスタンスを作成（モデル名とtop_nを指定） # model="rerank-multilingual-v3.0", top_n=top_n
	cohere_reranker=CohereRerank(model="rerank-multilingual-v3.0",top_n=top_n)
	# compress_documentsでリランキングを実行し、結果を返す # documents=documents, query=question
	return cohere_reranker.compress_documents(documents=documents,query=question)

# rerank_rag_chainを定義。RunnablePassthroughでquestionとdocumentsを渡し、rerankでcontextを追加し、プロンプト・モデル・出力パーサをパイプでつなぐ
rerank_rag_chain=(
	{
		"question":RunnablePassthrough(),
		"documents":retriever
	}
	|RunnablePassthrough.assign(context=rerank)
	|prompt|model|StrOutputParser()
)

# rerank_rag_chainに「LangChainの概要を教えて」という質問を投げる
rerank_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能があります。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築できます。LangGraphを利用することで、状態を持つエージェントを作成し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、さまざまなプロバイダーと統合し、標準化されたインターフェースを提供することで、開発者が異なるコンポーネントを簡単に切り替えたり、組み合わせたりできるようにします。また、複雑なアプリケーションのオーケストレーションをサポートするために、LangGraphというライブラリも提供しています。これにより、エージェントやマルチエージェントアプリケーションの構築が容易になります。'

## 6.5. 複数の Retriever を使う工夫


### LLM によるルーティング


In [ ]:
from langchain_community.retrievers import TavilySearchAPIRetriever  # TavilySearchAPIRetrieverのインポート

# retrieverに設定を付与してLangChainドキュメント用リトリーバーを作成
langchain_document_retriever=retriever.with_config(  # retrieverにrun_nameを設定
	{"run_name":"langchain_document_retriever"}  # run_nameを指定
)
# TavilySearchAPIRetrieverでウェブ検索用リトリーバーを作成し設定を付与
web_retriever=TavilySearchAPIRetriever(k=3).with_config(  # k=3で検索件数指定、run_nameを設定
	{"run_name":"web_retriever"}  # run_nameを指定
)

# Enumクラスをインポート（列挙型定義用）
from enum import Enum  # Enumのインポート

# どのリトリーバーを使うかを表す列挙型Routeを定義
class Route(str, Enum):  # 文字列型のEnumを継承
    langchain_document = "langchain_document"  # LangChainドキュメント用
    web = "web"  # ウェブ検索用

# モデル出力としてどのルートを選択したかを表すクラスRouteOutputを定義
class RouteOutput(BaseModel):  # BaseModelを継承
    route: Route  # routeフィールドにRoute型を指定

# Retriever選択用のプロンプトを作成
route_prompt=ChatPromptTemplate.from_template("""\  # テンプレートからプロンプト作成
質問に回答するために適切なRetrieverを選択してください。
質問: {question}
""")  # {question}を埋め込む

# プロンプト→モデル→出力整形のチェーンを作成
route_chain=(  # チェーンの定義
    route_prompt  # プロンプト
    |model.with_structured_output(RouteOutput)  # モデル出力をRouteOutputで構造化
    |(lambda x:x.route)  # routeフィールドのみ抽出
)

# ルートに応じて適切なリトリーバーを呼び出す関数を定義
def routed_retriever(inp:dict[str,Any])->list[Document]:  # 入力は辞書型、出力はDocumentリスト
	question=inp["question"]  # 質問文を取得
	route=inp["route"]  # ルートを取得

	if route==Route.langchain_document:  # LangChainドキュメント用の場合
		return langchain_document_retriever.invoke(question)  # LangChainドキュメントリトリーバーを呼び出し
	elif route==Route.web:  # ウェブ検索用の場合
		return web_retriever.invoke(question)  # ウェブリトリーバーを呼び出し

	raise ValueError(f"Unknown route: {route}")  # 未知のルートの場合は例外

# ルーティング付きRAGチェーンを定義
route_rag_chain=(  # チェーンの定義
	{
		"question":RunnablePassthrough(),  # 質問をそのまま渡す
		"route":route_chain  # ルートはroute_chainで決定
	}
	|RunnablePassthrough.assign(context=routed_retriever)  # contextにrouted_retrieverの結果を追加
	|prompt|model|StrOutputParser()  # プロンプト→モデル→文字列出力パーサ
)

<>:25: SyntaxWarning: invalid escape sequence '\ '
<>:25: SyntaxWarning: invalid escape sequence '\ '
/var/folders/sf/2870kdw54dbg3z6dyhtwdhwr0000gq/T/ipykernel_9258/1066379075.py:25: SyntaxWarning: invalid escape sequence '\ '
  route_prompt=ChatPromptTemplate.from_template("""\  # テンプレートからプロンプト作成


In [23]:
route_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下の3つの主要なステージがあります。\n\n1. **開発**: LangChainのオープンソースコンポーネントやサードパーティの統合を使用してアプリケーションを構築します。LangGraphを利用することで、状態を持つエージェントを作成し、ストリーミングや人間の介入をサポートします。\n\n2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできるようにします。\n\n3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換します。\n\nLangChainは、LLMや関連技術（埋め込みモデルやベクターストアなど）に対する標準インターフェースを実装し、数百のプロバイダーと統合しています。また、複数のオープンソースライブラリで構成されており、アーキテクチャやチュートリアル、APIリファレンスなどのリソースも提供されています。'

In [24]:
route_rag_chain.invoke("東京の今日の天気は？")

'東京の今日の天気は曇のち雨です。昼頃から所々で雨雲が湧き、夜には広く雨が降る予報です。'

### ハイブリッド検索の実装


In [30]:
from langchain_community.retrievers import BM25Retriever

chroma_retriever=retriever.with_config(
	{"run_name":"chroma_retriever"}
)
bm25_retriever=BM25Retriever.from_documents(documents).with_config(
	{"run_name":"bm25_retriever"}
)

from langchain_core.runnables import RunnableParallel

hybrid_retriever=(
	RunnableParallel({
		"chroma_documents":chroma_retriever,
		"bm25_documents":bm25_retriever,
	})
	|(lambda x:[x["chroma_documents"],x["bm25_documents"]])
	|reciprocal_rank_fusion
)
hybrid_rag_chain=(
	{
		"question":RunnablePassthrough(),
		"context":hybrid_retriever
	}
	|prompt|model|StrOutputParser()
)

hybrid_rag_chain.invoke("LangChainの概要を教えて")

'LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能があります：\n\n- **開発**: LangChainのオープンソースコンポーネントやサードパーティ統合を使用してアプリケーションを構築できます。LangGraphを利用することで、状態を持つエージェントを作成し、ストリーミングや人間の介入をサポートします。\n- **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。\n- **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。\n\nLangChainは、LLMや関連技術（埋め込みモデルやベクトルストアなど）に対する標準インターフェースを実装し、数百のプロバイダーと統合しています。また、複数のオープンソースライブラリで構成されており、さまざまな統合パッケージやコミュニティメンテナンスのリソースも提供しています。'